# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233891356578                   -0.50    7.0
  2   -7.249993647027       -1.79       -1.40    1.0
  3   -7.250920180465       -3.03       -1.81    4.0
  4   -7.250968942763       -4.31       -1.88    2.0
  5   -7.251331451140       -3.44       -2.66    2.0
  6   -7.251338115277       -5.18       -3.12    2.0
  7   -7.251338771062       -6.18       -3.82    2.0
  8   -7.251338795965       -7.60       -4.25    3.0
  9   -7.251338798617       -8.58       -5.01    2.0
 10   -7.251338798686      -10.16       -5.43    3.0
 11   -7.251338798704      -10.76       -6.03    3.0
 12   -7.251338798705      -12.03       -6.90    3.0
 13   -7.251338798705      -13.82       -7.14    3.0
 14   -7.251338798705      -14.57       -7.67    1.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.051475438983684806
[ Info: Arnoldi iteration step 2: normres = 0.42436363101309754
[ Info: Arnoldi iteration step 3: normres = 1.011707361747678
[ Info: Arnoldi iteration step 4: normres = 0.349547570318821
[ Info: Arnoldi iteration step 5: normres = 0.2152256289721488
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.50e-02, 2.57e-02, 1.50e-01, 1.14e-01, 9.69e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5527551681347113
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.13e-02, 8.62e-02, 5.29e-01, 9.13e-02, 2.03e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08882754968189606
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.97e-04, 4.09e-02, 8.53e-03, 2.83e-02, 7.12e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10384484477396469
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.65e-0